<h1>Face Detection and Privacy</h1>
This program focuses on creating an algorithm for detecting human faces under varying conditions, extracting the faces, and applying blurring to hide the eyes

In [ ]:
import cv2
import matplotlib.pyplot as plt

def load_image(image_path):
    return cv2.imread(image_path)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def detect_faces(gray_image):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    return face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

def detect_eyes(gray_image):
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    return eye_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

def detect_eyes_and_blur_faces(extracted_faces, eye_cascade_path='haarcascade_eye.xml'):
    eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

    blurred_faces = []
    for face_gray, face_color in extracted_faces:
        eyes = eye_cascade.detectMultiScale(face_gray, scaleFactor=1.1, minNeighbors=3, minSize=(15, 15))
        face_blurred = face_color.copy()
        for (ex, ey, ew, eh) in eyes:
            face_roi = face_blurred[ey:ey+eh, ex:ex+ew]
            blurred_roi = cv2.GaussianBlur(face_roi, (15, 15), 30)
            face_blurred[ey:ey+eh, ex:ex+ew] = blurred_roi
        blurred_faces.append(face_blurred)
    return blurred_faces



def process_image(image_path):
    image = load_image(image_path)
    gray_image = convert_to_grayscale(image)
    faces = detect_faces(gray_image)

    extracted_faces = []
    for (x, y, w, h) in faces:
        face_gray = gray_image[y:y+h, x:x+w]
        face_color = image[y:y+h, x:x+w]
        extracted_faces.append((face_gray, face_color))
    return extracted_faces


def draw_rectangles(image, faces, color=(0, 0, 255)):
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    return image


image_paths = ['musk_costume1.jpg', 'musk_distant.jpg', 'musk_full_body.jpg', 'musk_other_object.jpg', 'multiple_faces.jpg']

all_blurred_faces = []
for image_path in image_paths:
    extracted_faces = process_image(image_path)
    blurred_faces = detect_eyes_and_blur_faces(extracted_faces)
    all_blurred_faces.extend(blurred_faces)

# Display all images using matplotlib in one figure
fig, axes = plt.subplots(1, len(all_blurred_faces), figsize=(15, 5))
for ax, face in zip(axes, all_blurred_faces):
    ax.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
    ax.axis('off')
plt.show()


In [40]:
import cv2
import matplotlib.pyplot as plt
from mtcnn import MTCNN

def extract_faces(image, faces):
    extracted_faces = []
    for face in faces:
        x, y, w, h = face['box']
        extracted_face = image[y:y + h, x:x + w]
        extracted_faces.append(extracted_face)
    return extracted_faces

def blur_eyes(face, keypoints):
    left_eye_x, left_eye_y = keypoints['left_eye']
    right_eye_x, right_eye_y = keypoints['right_eye']
    eye_width = int(abs(left_eye_x - right_eye_x) * 0.5)
    
    left_eye_area = face[left_eye_y - eye_width:left_eye_y + eye_width, left_eye_x - eye_width:left_eye_x + eye_width]
    right_eye_area = face[right_eye_y - eye_width:right_eye_y + eye_width, right_eye_x - eye_width:right_eye_x + eye_width]
    
    left_eye_blurred = cv2.GaussianBlur(left_eye_area, (15, 15), 30)
    right_eye_blurred = cv2.GaussianBlur(right_eye_area, (15, 15), 30)
    
    face[left_eye_y - eye_width:left_eye_y + eye_width, left_eye_x - eye_width:left_eye_x + eye_width] = left_eye_blurred
    face[right_eye_y - eye_width:right_eye_y + eye_width, right_eye_x - eye_width:right_eye_x + eye_width] = right_eye_blurred
    
    return face

def process_image(image_path, mtcnn_detector):
    image = cv2.imread(image_path)
    detected_faces = mtcnn_detector.detect_faces(image)
    
    extracted_faces = extract_faces(image, detected_faces)

    processed_faces = []
    for face, detected_face in zip(extracted_faces, detected_faces):
        blurred_eyes_face = blur_eyes(face, detected_face['keypoints'])
        processed_faces.append(blurred_eyes_face)

    return processed_faces

# Initialize MTCNN detector
mtcnn_detector = MTCNN()

# Process multiple images
image_paths = ['musk_costume1.jpg', 'musk_distant.jpg', 'musk_full_body.jpg', 'musk_other_object.jpg', 'multiple_faces.jpg']
all_processed_faces = [process_image(image_path, mtcnn_detector) for image_path in image_paths]

# Display extracted faces with blurred eyes by saving images to disk
for i, processed_faces in enumerate(all_processed_faces):
    for j, face in enumerate(processed_faces):
        plt.figure()
        plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        plt.title(f'Image {i + 1}, Face {j + 1}')
        plt.axis('off')
        plt.savefig(f'output_image_{i + 1}_face_{j + 1}.png') # Save the image to disk
        plt.close()


1/1 [==============================] - 0s 93ms/step


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'
